In [ ]:
import os
import sys
import json
import urllib
import pandas as pd
from urllib import parse, request

In [ ]:
REST_URL = "http://data.bioontology.org"
API_KEY = '3b00793b-f3cc-489c-9d6e-7a888f4b656e'

In [ ]:
def parse_directory(path_to_directory):
    '''documentation here'''
    global path_files
    path_files = []

    for root, dirs, files in os.walk(path_to_directory):
        for file in files:
            path_files.append(os.path.join(root, file))

    return path_files

In [ ]:
parse_directory('data/kaggle/')

In [ ]:
def selec_ontologies(path_to_select_ontologies):
    '''read file which contains a list of ontlogies selected by researchers'''
    
    ont_sel = pd.read_csv('data/ont_select.csv')

    list_onto = [x.replace('\t','').replace(' ','').replace('\n','') for x in list(ont_sel['Ontology'].values)if type(x)!=float]
    
    global onto_select
    onto_select= []
    
    for i in list_onto:
        
        if i not in onto_select:
            onto_select.append(i)
        else:
            next
            
        return onto_select

In [ ]:
select_ontologies('data/ont_select.csv')

In [ ]:
def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

In [ ]:
def bio_onts():
    '''retrieve all ontologies stored in bioportal'''
    
    resources = get_json(REST_URL + "/")
    ontologies = get_json(resources["links"]["ontologies"])
    
    global onto_bio
    onto_bio = []
    for ontology in ontologies:
        onto_bio.append(f"{ontology['name']}\n{ontology['@id']}\n")
        
    return onto_bio    

In [ ]:
bio_onts()

In [ ]:
def ontologies():
    '''merge ontologies selected and ontologies from bio portal'''
    
    onto_bio = bio_onts()
    onto_select = select_ontologies('data/ont_select.csv')
    
    
    global ontologies
    ontologies = '' 
    
    for ontosel in onto_select:
        for ontobio in onto_bio:
            if ontobio.find(ontosel)>1:
                if not ontologies.find(ontosel)>1:
                    ontologies += ontosel +' '
                else:
                    next
    ontologies = ontologies[:-1].replace(' ',',')
    
    return ontologies

In [ ]:
ontologies()

In [ ]:
def make_url_query(API_KEY,
                   ontologies, 
                   expand_class_hierachy_text,
                   class_hierachy_max_level, text):
    global url
    url = REST_URL+ """/annotator/?apikey={}&ontologies={}&expand_class_hierachy={}&class_hierachy_max_level={}&text={}""".format(API_KEY, ontologies,
                                          expand_class_hierachy_text,class_hierachy_max_level,text)
    return url

In [ ]:
def call_api_to_annotate(text_to_annotate):
    '''documentation here'''
    
    expand_class_hierachy='true'
    class_hierachy_max_level=3
    
    url= make_url_query(API_KEY, ontologies, expand_class_hierachy, class_hierachy_max_level, parse.quote(text_to_annotate))
    api_call = request.urlopen(url)
    global annotations
    annotations = json.loads(api_call.read().decode('utf-8'))
    return annotations

In [ ]:
class annotationsdb:
    '''
    this class store data for each record annotated;
    public variable: dbDict which contain data;
    private variable: none;
    methods: - for update dict values;
             - for store data in to json file;
    '''
    
    db = {'paper_idx':[],'paper_id':[], 'title':[], 'annotations':[]}
    
    def __init__(self):
        pass
          
    def updateValues(valueIdx, valueId, valueTitle, valueAnnotations):
        annotationsdb.db['paper_idx'].append(valueIdx)
        annotationsdb.db['paper_id'].append(valueId)
        annotationsdb.db['title'].append(valueTitle)
        annotationsdb.db['annotations'].append(valueAnnotations)
        
        if len(annotationsdb.db['annotations']):
            return 'update OK'
        
    def storeData(self):
        with open('annotatedPapersDB.json','w') as file_handler:
            json.dump(db, file_handler)
            
    def __str__():
        print(annotationsdb.db)

In [ ]:
sam_r1=paths[0]  # pmc 

In [ ]:
sam_r2=paths[48] #pdf_json
sam_r2

In [ ]:
list_paths = [sam_r1,sam_r2]

In [ ]:
with open(sam_r1,'r') as json_handler:
    data1 = json.load(json_handler)

In [ ]:
data1.keys()

In [ ]:
#data['paper_id']            ####pmc
#data['metadata']['title']
#data['body_text'] list of paragraphs
#data1['body_text'][17]

In [ ]:
with open(sam_r2,'r') as json_handler:
    data2 = json.load(json_handler)

In [ ]:
data2.keys()

In [ ]:
len(data2['body_text'])

In [ ]:
#data['paper_id']            #### pdf_json
#data['metadata']['title']
#data['abstract'] list of paragraps 
#data['body_text'] list of paragraps

###  sintesis of above things: case to papers: one pmc one pdf_json

In [ ]:
idx = 0
for path in list_paths:
    
    list_annotations = []
    
    with open(path,'r') as json_handler:
        data = json.load(json_handler)
        
    try: 
        for paragraph in data['abstract']:
            text_to_annotate = paragraph['text']
            
            annotations = call_api_to_annotate(text_to_annotate)
            print(len(annotations))
            if annotations:
                for annotation in anottations:
                    list_annotations.append(annotation)
            else:
                pass
                 
        
        for paragraph in data['body_text']:
            tex_to_annotate = paragraph['text']
            
            annotations = call_api_to_annotate(tex_to_annotate)
            
            print(len(annotations))
            if annotation:
                for annotation in annotations:
                    list_anotations.append(annotation)
                else:
                    pass
            
    except:
        p = 0
        for paragraph in data['body_text']:
            text_to_annotate = paragraph['text']
            
            if text_to_annotate:
            
                annotations = call_api_to_annotate(text_to_annotate)
                print(len(annotations))

                if annotations:
                    for annotation in annotations:

                        list_annotations.append(annotation)
                else: 
                    pass  
            else:
                pass
            p +=1
                
    finally:
        #store annotations in list anotations
        text_to_annotate = data['metadata']['title']
        
        #call api
        annotations = call_api_to_annotate(text_to_annotate)
        print(len(annotations))
        print('finally')
        for annotation in annotations:
            list_annotations.append(annotation)
            
    annotationsdb.updateValues(idx, data['paper_id'], data['metadata']['title'], list_annotations)
    idx +=1

In [ ]:
382+127+296+228+43+212+65+74+637+255+310+864+185+419+99+168+33+62

In [ ]:
len(annotationsdb.db['annotations'][0])